In [1]:
### If on Ubuntu, run in CLI:
# sudo apt install build-essential libpoppler-cpp-dev pkg-config python3-dev

In [2]:
%pip install -r requirements.txt

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.2.13/MindSpore/unified/x86_64/mindspore-2.2.13-cp39-cp39-linux_x86_64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple
%pip install mindnlp

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.2.13/MindSpore/unified/x86_64/mindspore-2.2.13-cp39-cp39-linux_x86_64.whl (756.2 MB)
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [3]:
### If doesn't work, run in CLI
# !pip install -U huggingface_hub
# !export HF_ENDPOINT=https://hf-mirror.com
# !huggingface-cli download --resume-download BAAI/bge-large-zh-v1.5 --local-dir /home/models/bge-large-zh-v1.5
# model_path = '/home/models/bge-large-zh-v1.5'

In [4]:
### Preprocess pdf documents
import pdftotext
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema.document import Document

# modified from https://stackoverflow.com/questions/77045559/langchain-load-with-string
def get_text_chunks_langchain(text):
    """ Turns raw string into docs that conform with docs = loader.load()"""
    text_splitter = CharacterTextSplitter(
		separator="\n",
		chunk_size=1000,
		chunk_overlap=50,
		length_function=len
	)
    docs = [Document(page_content=x) for x in text_splitter.split_text(text)]
    return docs

def load_pdf(filepath):
    """ From a pdf, return a docs"""
    text = ""
    with open(filepath, "rb") as f:
        pdf = pdftotext.PDF(f)
        for page in pdf:
            text += page
    return get_text_chunks_langchain(text)


""" Unit test """
pdf_files = ["./data/CMA_ES.pdf", "./data/SBX.pdf"]
documents = []

for pdf_file in pdf_files:
    docs = load_pdf(pdf_file)

print(docs[0])

page_content='arXiv:1604.00772v2 [cs.LG] 10 Mar 2023\nThe CMA Evolution Strategy: A Tutorial\nNikolaus Hansen\nInria\nResearch centre Saclay–Île-de-France\nContents\nNomenclature\n2\n0\nPreliminaries\n0.1 Eigendecomposition of a Positive Definite Matrix . . . . . . . . . . . . . . .\n0.2 The Multivariate Normal Distribution . . . . . . . . . . . . . . . . . . . . .\n0.3 Randomized Black Box Optimization . . . . . . . . . . . . . . . . . . . . .\n0.4 Hessian and Covariance Matrices . . . . . . . . . . . . . . . . . . . . . . . .\n3\n4\n5\n6\n7\n1\nBasic Equation: Sampling\n8\n2\nSelection and Recombination: Moving the Mean\n8\n3\nAdapting the Covariance Matrix\n3.1 Estimating the Covariance Matrix From Scratch . . . . . . . . . . . . . . . .\n3.2 Rank-µ-Update . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n3.3 Rank-One-Update . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n3.3.1 A Different Viewpoint . . . . . . . . . . . . . . . . . . . . . . . . .

In [6]:
### Load model from local files
from transformers import AutoModel, AutoTokenizer

# model_path = '/home/models/bge-large-zh-v1.5'
model_path = './bge-large-zh-v1.5'

model = AutoModel.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

/home/nore/miniconda3/envs/mindspore/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
### Embed documents into vectordb
from langchain.vectorstores import Chroma
from sentence_transformers import SentenceTransformer
from FlagEmbedding import BGEM3FlagModel
from langchain.embeddings import HuggingFaceBgeEmbeddings

embeddings = HuggingFaceBgeEmbeddings(model_name=model_path)

vector_store = Chroma(embedding_function=embeddings)
vector_store.add_documents(docs)
retriever = vector_store.as_retriever()

""" Unit test """
query = "What is the main topic of the document?"
retrieved_docs = retriever.get_relevant_documents(query)
for doc in retrieved_docs:
    print(doc)

In [9]:
# for doc in retrieved_docs:
#     print(doc)

page_content='recherches, Université Paris-Sud, 2010.\n[18] Hansen N. Injecting External Solutions Into CMA-ES. CoRR, arXiv:1110.4181, 2011.\n[19] Hansen N, Auger A. Principled design of continuous stochastic search: From theory\nto practice. In Y Borenstein and A Moraglio, eds.: Theory and Principled Methods for\nDesigning Metaheustics. Springer, pages 145–180, 2014.\n[20] Hansen N, Atamna A, Auger A. How to Assess Step-Size Adaptation Mechanisms in\nRandomised Search. In Parallel Problem Solving from Nature – PPSN XIII, pages 60–69.\nSpringer, 2014.\n[21] Hansen N, Kern S. Evaluating the CMA evolution strategy on multimodal test functions.\nIn Xin Yao et al., editors, Parallel Problem Solving from Nature – PPSN VIII, pages\n282–291. Springer, 2004.\n[22] Hansen N, Niederberger SPN, Guzzella L, Koumoutsakos P. A method for handling\nuncertainty in evolutionary optimization with an application to feedback control of combustion. IEEE Transactions on Evolutionary Computation, 13(1):180–

In [2]:
import argparse
import hashlib
import os
import re
from threading import Thread
from typing import Union, List

import jieba
from loguru import logger
from mindnlp.peft import PeftModel
from msimilarities import (
    EnsembleSimilarity,
    BertSimilarity,
    BM25Similarity,
)
from msimilarities.similarity import SimilarityABC
from mindnlp.transformers import (
    AutoModel,
    AutoModelForCausalLM,
    AutoTokenizer,
    BloomForCausalLM,
    BloomTokenizerFast,
    LlamaTokenizer,
    LlamaForCausalLM,
    TextIteratorStreamer,
    GenerationConfig,
    AutoModelForSequenceClassification,
)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.516 seconds.
Prefix dict has been built successfully.
2024-05-28 11:52:42.140017: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-28 11:52:42.650593: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-28 11:52:44.456398: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
MODEL_CLASSES = {
    "bloom": (BloomForCausalLM, BloomTokenizerFast),
    "chatglm": (AutoModel, AutoTokenizer),
    "llama": (LlamaForCausalLM, LlamaTokenizer),
    "baichuan": (AutoModelForCausalLM, AutoTokenizer),
    "auto": (AutoModelForCausalLM, AutoTokenizer),
}

PROMPT_TEMPLATE = """基于以下已知信息，简洁和专业的来回答用户的问题。
如果无法从中得到答案，请说 "根据已知信息无法回答该问题" 或 "没有提供足够的相关信息"，不允许在答案中添加编造成分，答案请使用中文。

已知内容:
{context_str}

问题:
{query_str}
"""

In [4]:
print("hi")

hi


In [ ]:
from mindnlp.transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import gradio as gr
# import mdtex2html

model = AutoModelForSeq2SeqLM.from_pretrained('./chatglm3-6b')
model.set_train(False)
tokenizer = AutoTokenizer.from_pretrained('./chatglm3-6b')

: 

In [6]:
prompt = '你好'
history = []
response, _ = model.chat(tokenizer, prompt, history=history, max_length=20)
response

/home/nore/miniconda3/envs/mindspore/lib/python3.9/site-packages/mindnlp/transformers/generation/utils.py:1402: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://hf-mirror.com/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
The dtype of attention mask (Float32) is not bool


: 

In [3]:
# class ChatBabel:
#     def __init__(
#             self,
#             enable_history: bool = False,
#             generate_model_type: str = "chatglm",
#             generate_model_name_or_path: str = "THUDM/chatglm3-6b",
#             lora_model_name_or_path: str = None,
#             int8: bool = False,
#             int4: bool = False,
#     ):
#         """
#         Init RAG model.
#         :param similarity_model: similarity model, default None, if set, will use it instead of EnsembleSimilarity
#         :param generate_model_type: generate model type
#         :param generate_model_name_or_path: generate model name or path
#         :param lora_model_name_or_path: lora model name or path
#         :param corpus_files: corpus files
#         :param save_corpus_emb_dir: save corpus embeddings dir, default ./corpus_embs/
#         :param int8: use int8 quantization, default False
#         :param int4: use int4 quantization, default False
#         :param chunk_size: chunk size, default 250
#         :param chunk_overlap: chunk overlap, default 0, can not set to > 0 if num_expand_context_chunk > 0
#         :param rerank_model_name_or_path: rerank model name or path, default 'BAAI/bge-reranker-base'
#         :param enable_history: enable history, default False
#         :param num_expand_context_chunk: num expand context chunk, default 2, if set to 0, will not expand context chunk
#         :param similarity_top_k: similarity_top_k, default 5, similarity model search k corpus chunks
#         :param rerank_top_k: rerank_top_k, default 3, rerank model search k corpus chunks
#         """
#         self.history = []
#         self.gen_model, self.tokenizer = self._init_gen_model(
#             generate_model_type,
#             generate_model_name_or_path,
#             peft_name=lora_model_name_or_path,
#             int8=int8,
#             int4=int4,
#         )

#     def __str__(self):
#         return f"Similarity model: {self.sim_model}, Generate model: {self.gen_model}"
    
#     def _init_gen_model(
#             self,
#             gen_model_type: str,
#             gen_model_name_or_path: str,
#             peft_name: str = None,
#             int8: bool = False,
#             int4: bool = False,
#     ):
#         """Init generate model."""
#         model_class, tokenizer_class = MODEL_CLASSES[gen_model_type]
#         tokenizer = tokenizer_class.from_pretrained(gen_model_name_or_path, mirror='modelscope')
#         model = model_class.from_pretrained(
#             gen_model_name_or_path, mirror='modelscope'
#         )
#         try:
#             model.generation_config = GenerationConfig.from_pretrained(gen_model_name_or_path, mirror='modelscope')
#         except Exception as e:
#             logger.warning(f"Failed to load generation config from {gen_model_name_or_path}, {e}")
#         if peft_name:
#             model = PeftModel.from_pretrained(
#                 model,
#                 peft_name,
#             )
#             logger.info(f"Loaded peft model from {peft_name}")
#         model.set_train(False)
#         return model, tokenizer
    
#     def _get_chat_input(self):
#         messages = []
#         for conv in self.history:
#             if conv and len(conv) > 0 and conv[0]:
#                 messages.append({'role': 'user', 'content': conv[0]})
#             if conv and len(conv) > 1 and conv[1]:
#                 messages.append({'role': 'assistant', 'content': conv[1]})
#         input_ids = self.tokenizer.apply_chat_template(
#             conversation=messages,
#             tokenize=True,
#             add_generation_prompt=True,
#             return_tensors='ms'
#         )
#         return input_ids
    
#     def stream_generate_answer(
#             self,
#             max_new_tokens=512,
#             temperature=0.7,
#             repetition_penalty=1.0,
#             context_len=2048
#     ):
#         streamer = TextIteratorStreamer(self.tokenizer, timeout=60.0, skip_prompt=True, skip_special_tokens=True)
#         input_ids = self._get_chat_input()
#         max_src_len = context_len - max_new_tokens - 8
#         input_ids = input_ids[-max_src_len:]
#         generation_kwargs = dict(
#             input_ids=input_ids,
#             max_new_tokens=max_new_tokens,
#             temperature=temperature,
#             do_sample=True,
#             repetition_penalty=repetition_penalty,
#             streamer=streamer,
#         )
#         thread = Thread(target=self.gen_model.generate, kwargs=generation_kwargs)
#         thread.start()

#         yield from streamer
    

#     def predict_stream(
#             self,
#             query: str,
#             max_length: int = 512,
#             context_len: int = 2048,
#             temperature: float = 0.7,
#     ):
#         """Generate predictions stream."""
#         stop_str = self.tokenizer.eos_token if self.tokenizer.eos_token else "</s>"
#         if not self.enable_history:
#             self.history = []
#         if self.sim_model.corpus:
#             reference_results = self.get_reference_results(query)
#             if not reference_results:
#                 yield '没有提供足够的相关信息', reference_results
#             reference_results = self._add_source_numbers(reference_results)
#             context_str = '\n'.join(reference_results)[:(context_len - len(PROMPT_TEMPLATE))]
#             prompt = PROMPT_TEMPLATE.format(context_str=context_str, query_str=query)
#             logger.debug(f"prompt: {prompt}")
#         else:
#             prompt = query
#             logger.debug(prompt)
#         self.history.append([prompt, ''])
#         response = ""
#         for new_text in self.stream_generate_answer(
#                 max_new_tokens=max_length,
#                 temperature=temperature,
#                 context_len=context_len,
#         ):
#             if new_text != stop_str:
#                 response += new_text
#                 yield response
    
#     def predict(
#             self,
#             query: str,
#             max_length: int = 512,
#             context_len: int = 2048,
#             temperature: float = 0.7,
#     ):
#         """Query from corpus."""
#         reference_results = []
#         if not self.enable_history:
#             self.history = []
#         if self.sim_model.corpus:
#             reference_results = self.get_reference_results(query)

#             if not reference_results:
#                 return '没有提供足够的相关信息', reference_results
#             reference_results = self._add_source_numbers(reference_results)
#             context_str = '\n'.join(reference_results)[:(context_len - len(PROMPT_TEMPLATE))]
#             prompt = PROMPT_TEMPLATE.format(context_str=context_str, query_str=query)
#             logger.debug(f"prompt: {prompt}")
#         else:
#             prompt = query
#         self.history.append([prompt, ''])
#         response = ""
#         for new_text in self.stream_generate_answer(
#                 max_new_tokens=max_length,
#                 temperature=temperature,
#                 context_len=context_len,
#         ):
#             response += new_text
#         response = response.strip()
#         self.history[-1][1] = response
#         return response, reference_results

In [5]:
# m = ChatBabel(
#         enable_history = False,
#         generate_model_type = "chatglm",
#         generate_model_name_or_path = "./chatglm3-6b",
#         lora_model_name_or_path = None,
#         int8 = False,
#         int4 = False
# )


AssertionError: 